In [1]:
import tensorflow as tf
import keras
print(tf.__version__)

2.16.1


In [2]:
base_dir = './Jute_Pest_Dataset'
batch_size = 32
img_height = 180
img_width = 180

train_dir = f'{base_dir}/train'
val_dir = f'{base_dir}/val'
test_dir = f'{base_dir}/test'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)
# Save the class names for later use
class_names = train_ds.class_names

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_dir,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2085 files belonging to 5 classes.
Found 171 files belonging to 5 classes.
Found 116 files belonging to 5 classes.


In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),

])


In [4]:
for images, labels in train_ds.take(1):  # Taking a single batch
    print(images.numpy().shape)  # Accessing the feature tensors (images)
    print(labels.numpy())  # Accessing the integer labels
    print([class_names[label] for label in labels.numpy()])  # Convert integer labels to class names


(32, 180, 180, 3)
[1 0 3 2 0 0 1 2 1 1 3 4 0 4 0 2 2 3 2 0 1 4 3 1 3 3 2 3 1 0 3 3]
['Leaf Beetle', 'Jute Stem Weevil', 'Termite', 'Mealybug', 'Jute Stem Weevil', 'Jute Stem Weevil', 'Leaf Beetle', 'Mealybug', 'Leaf Beetle', 'Leaf Beetle', 'Termite', 'Yellow Mite', 'Jute Stem Weevil', 'Yellow Mite', 'Jute Stem Weevil', 'Mealybug', 'Mealybug', 'Termite', 'Mealybug', 'Jute Stem Weevil', 'Leaf Beetle', 'Yellow Mite', 'Termite', 'Leaf Beetle', 'Termite', 'Termite', 'Mealybug', 'Termite', 'Leaf Beetle', 'Jute Stem Weevil', 'Termite', 'Termite']


In [5]:
from functools import partial
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=[180, 180, 3])
# freeze the base model to prevent its weights from being updated during training
base_model.trainable = False

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    keras.layers.Input(shape=[180,180,3]),
    data_augmentation,
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [6]:
model.summary() 

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 6, 6, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,858,890 (91.01 MB)

 Trainable params: 271,178 (1.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 56s 764ms/step - accuracy: 0.3626 - loss: 1.9638 - val_accuracy: 0.8480 - val_loss: 0.4564
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 50s 752ms/step - accuracy: 0.7394 - loss: 0.7228 - val_accuracy: 0.8830 - val_loss: 0.3679
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 49s 737ms/step - accuracy: 0.8415 - loss: 0.4379 - val_accuracy: 0.9298 - val_loss: 0.2688
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 49s 741ms/step - accuracy: 0.8939 - loss: 0.2912 - val_accuracy: 0.8947 - val_loss: 0.4207
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 49s 747ms/step - accuracy: 0.9246 - loss: 0.2689 - val_accuracy: 0.9474 - val_loss: 0.3187
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 53s 811ms/step - accuracy: 0.9210 - loss: 0.2116 - val_accuracy: 0.9357 - val_loss: 0.2721
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 56s 843ms/step - accuracy: 0.9431 - loss: 0.1649 - val_accuracy: 0.9415 - val_loss: 0.2787
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 49s 747ms/step - accuracy: 0.9569 - loss: 0.1282 - val_accu